In [1]:
import os
os.chdir("../")

In [2]:
import time
import openai
import numpy as np
from mega.data.load_datasets import load_xnli_dataset
from mega.data.data_utils import choose_few_shot_examples
from mega.prompting.instructions import INSTRUCTIONS
from mega.prompting.prompting_utils import load_prompt_template
from mega.utils.env_utils import load_openai_env_variables
from mega.models.completion_models import get_model_pred, gpt3x_completion
from mega.prompting.prompting_utils import construct_prompt
from tqdm import tqdm

In [3]:
# Make sure that {env_name}.env file is present in the envs/ directory
env_name = "melange"
load_openai_env_variables()

In [4]:
openai.api_base

'https://gpttesting1.openai.azure.com/'

In [5]:
model = "gpt-35-turbo"
pivot_lang = "hi"
tgt_lang = "hi"
prompt_name = "GPT-3 style"
few_shot_k = 8

In [6]:
# Loading datasets
train_dataset = load_xnli_dataset(pivot_lang, split = "train")
test_dataset = load_xnli_dataset(tgt_lang, split = "validation")

Found cached dataset xnli (/home/t-kabirahuja/.cache/huggingface/datasets/xnli/hi/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset xnli (/home/t-kabirahuja/.cache/huggingface/datasets/xnli/hi/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# Loading prompt template
prompt_template = load_prompt_template(pivot_lang, prompt_name, dataset = "xnli")
print(prompt_template.jinja)

{{premise}}
Question: {{hypothesis}} True, False, or Neither? ||| {{ answer_choices[label] }}


In [8]:
# Loading instruction for the task
instruction = INSTRUCTIONS["xnli"]
print(instruction)

You are an NLP assistant whose purpose is to solve Natural Language Inference (NLI) problems. NLI is the task of determining the inference relation between two (short, ordered) texts: entailment, contradiction, or neutral. Answer as concisely as possible in the same format as the examples below:


In [9]:
# Getting few-shot examples
train_examples = choose_few_shot_examples(
        train_dataset, few_shot_k, selection_criteria="random")

In [13]:
test_example = test_dataset[0]

prompt, label = construct_prompt(
    train_examples,
    test_dataset[0],
    train_prompt_template=prompt_template,
    test_prompt_template=prompt_template,
    chat_prompt=False,
    instruction=instruction
)
prompt

'लेकिन 1990 के दशक में संघीय सरकार के आगमन के साथ भी , नए निवेश के लिए राष ् ट ् रीय बचत काफी कम होती है क ् योंकि व ् यक ् तिगत बचत में नाटकीय रूप से गिरावट आई है .\nQuestion: व ् यक ् तिगत बचत में नाटकीय वृद ् धि हुई . True, False, or Neither?\nFalse\nEminene से नैया गुजरता है ? ? z kulesi ( प ् रथम का टावर ) , 200 मीटर ( 600 फुट ) अपतटीय के बारे में एक छोटे द ् वीप पर स ् थित है .\nQuestion: Eminene से नैया में एफिल टॉवर की ओर स ् थित है , एक बड ़ े द ् वीप पर स ् थित है . True, False, or Neither?\nFalse\nसंयुक ् त राज ् य अमेरिका के हाल ही में संयुक ् त राज ् य पैरोल बोर ् ड से रोका गया है कि इंटरनेट का उपयोग करने के लिए इंटरनेट का उपयोग करने के लिए , किताबों , पत ् रिकाएं और समाचार पत ् रों को भी रोकने के लिए विस ् तृत किया जाना चाहिए .\nQuestion: संघीय कैदी जो पैरोल प ् राप ् त करने की अनुमति नहीं है उन ् हें इंटरनेट का उपयोग करने की अनुमति नहीं है और उन ् हें अपने स ् मार ् टफ ़ ोन की अनुमति नहीं है . True, False, or Neither?\nNeither\nफिर भी , उसने अपनी सामान ् य गति के निकट पा

In [14]:
prediction = gpt3x_completion(
    prompt,
    model,
    temperature=0,
    max_tokens=10
)
match = float(prediction.startswith(label))
print(f"Prediction: {prediction}")
print(f"Label: {label}")
print(f"Match: {match}")

Prediction: True
Label: Neither
Match: 0.0


Bad pipe message: %s [b"\xa5\xc2\xe6TP\xc1\x17$Q\x12K\x80\x0eB\x1d<o\xe1\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00"]
Bad pipe message: %s [b'\x8d\xd9\x8e\x96y\t\x93\xecv\x8a\xb5\xac\xa7!C\xdc\xbd|\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc', b"\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x0

In [12]:
openai.api_version

'2022-12-01'

In [13]:
matches = []
preds = []
labels = []
for test_example in tqdm(test_dataset.select(range(100))):
    prompt, label = construct_prompt(
        train_examples,
        test_example,
        train_prompt_template=prompt_template,
        test_prompt_template=prompt_template,
        chat_prompt=True,
        instruction=instruction
    )
    prediction = gpt3x_completion(
        prompt,
        model,
        temperature=0,
        max_tokens=10
    )
    time.sleep(1/2)
    match = float(prediction.startswith(label))
    preds.append(prediction)
    labels.append(label)
    matches.append(match)

print(f"Accuracy: {np.mean(matches)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:32<00:00,  1.09it/s]

Accuracy: 0.54
